## 复习一下psedo inverse,pinv的用处：
$M^{*}*M=I(m>=n)$

$M^{*}*M \approx I(m<n)$

In [1]:
import numpy as np

In [2]:
def pinv(M):
    U,S,VT=np.linalg.svd(M,False)
    
    linv=(VT.T*(1/S)).dot(U.T)
    return linv;
def check(a,b,eps=1e-5):
    assert np.abs(a-b).max()<eps,"计算错误"

In [3]:
A=np.random.randn(7,9)
Ainv=pinv(A)
Ainv_np=np.linalg.pinv(A)
check(Ainv,Ainv_np)

### solve:非线性方程(牛顿法)
$$f(x)=0$$
$$f(x)\approx f(x_0)+\nabla f(x_0) (x-x_0)=0$$
$$x=x_0-\nabla f(x_0)^{-1} f(x_0)$$
其中：$\nabla f(x_0)^{-1}$可以使用pinv代替
$$\nabla f_{i,j}=\frac{\partial f_i}{\partial x_j}$$

In [10]:
class Functional():
    def setParameter(self,p):
        self.p=p
    def fn(self):
        raise NotImplemented
    def grad_fn(self):
        raise NotImplemented
class LinearFunction(Functional):
    def __init__(self,M,N):
        self.A=np.random.randn(M,N)
        self.bias=np.random.randn(M,1)*np.random.randn(M,1)+2
    def reset(self):
        self.p=np.random.randn(self.A.shape[1],1)
        return self.p
    def __call__(self):
        return self.A.dot(self.p)+self.bias
    def grad_fn(self):
        return self.A
class CosFunction(Functional):
    def __init__(self,M,N):
        self.A=np.random.randn(M,N)
        cos_result=np.random.rand(N,1)
        self.bias=-self.A.dot(cos_result)
    def reset(self):
        self.p=np.random.randn(self.A.shape[1],1)
        return self.p
    def __call__(self):
        return self.A.dot(np.cos(self.p))+self.bias
    def grad_fn(self):
        h=1e-5
        grad=np.zeros_like(self.A)
    
        oldp=self.p.copy()
        
        for i in range(len(self.p)):
            self.p[i]=oldp[i]+h
            forward=self.__call__().ravel()
            self.p[i]=oldp[i]-h
            backward=self.__call__().ravel()
            self.p[i]=oldp[i]
            grad[:,i]=(forward-backward)/(2*h)
        
        self.p=oldp
        return grad
    
def solver(f,eps=1e-3,maxIters=100,verbose=False):
    x0=f.reset()
    for t in range(maxIters+1):
        fn=f()   
        err=np.abs(fn).max()
        if err<eps:
            break
        g=f.grad_fn()
        x0=x0-np.linalg.pinv(g).dot(fn)
        f.setParameter(x0)
        
        if verbose and t%1==0:
            print('err %.4f,grad=%f'%(err,np.linalg.norm(g,'fro')))
    print("run %d iters,max error %.4f,grad=%f"%(t,err,np.linalg.norm(g,'fro')))

In [13]:
fn1=LinearFunction(3,3)
fn2=LinearFunction(4,3)
fn3=CosFunction(3,3)
# solver(fn1)
# solver(fn2)
solver(fn3,verbose=True)

err 0.3824,grad=1.675102
err 0.2696,grad=1.956923
err 0.0063,grad=1.957118
run 3 iters,max error 0.0000,grad=1.957118


In [14]:
fn3()

array([[-3.84246167e-07],
       [-3.53815740e-06],
       [ 2.01893911e-06]])